In [1]:
import tensorflow as tf
import stackeddag.tf as sd

/home/junji-hashimoto/git/py-stacked-dag/.tox/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
calc_g = tf.Graph()

In [3]:
with calc_g.as_default():
    a = tf.constant(1, name="a")
    b = tf.constant(2, name="b")
    c = tf.add(a, b, name="add")

In [4]:
print(sd.fromGraph(calc_g),end="")

o o    a,b
|/
o    add


In [5]:
reg_g = tf.Graph()

In [6]:
with reg_g.as_default():
    import numpy as np
    x_data = np.random.rand(100).astype(np.float32)
    y_data = x_data * 0.1 + 0.3
    W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
    b = tf.Variable(tf.zeros([1]))
    y = W * x_data + b
#    loss = tf.reduce_mean(tf.square(y - y_data))
#    optimizer = tf.train.GradientDescentOptimizer(0.5)
#    train = optimizer.minimize(loss)

In [7]:
print(sd.fromGraph(reg_g),end="")

o o o    random_uniform/{max,min,shape}
 \|x
 /|  \
o o o |    random_uniform/RandomUniform,random_uniform/sub,Variable
|/ / \ \
| |    \ \
o o o o | |    random_uniform/mul,Variable/read,Variable_1,mul/y
| | |x __x
| |/|/ \  |
| |/|   | |
|/| |   | |
o o o o | |    random_uniform,mul,Variable_1/read,zeros
| |/ /_/ /
| | |__/
| |/|
|/| |
o o o    Variable/Assign,add,Variable_1/Assign


In [8]:
mnist_g = tf.Graph()

In [9]:
with mnist_g.as_default():
    x = tf.placeholder(tf.float32, [None, 784])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    y_ = tf.placeholder(tf.float32, [None, 10])
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [10]:
print(sd.fromGraph(mnist_g),end="")

o    Variable
|\
|  \
|    \
o o o |    Variable/read,Placeholder,Variable_1
|x x \ \
| |\ \ \ \
o o | | | |    MatMul,Variable_1/read
|x  | | | |
o | | | | |    add
| | | | | |
o | | | | |    Softmax
|\ \ \ \ \ \
|  \ \ \ \ \ \
o o | | | | | |    Log,Placeholder_1
|x \ \ \ \ \ \ \
|  \ \ \ \ \ \ \ \
o o | | | | | | | |    mul,Sum/reduction_indices
|x \ \ \ \ \ \ \ \ \
o | | | | | | | | | |    Sum
|  \ \ \ \ \ \ \ \ \ \
|    \ \ \ \ \ \ \ \ \ \
o o o | | | | | | | | | |    Neg,gradients/Mean_grad/Const_1,gradients/Mean_grad/Shape_2
|\|/   \ \ \ \ \ \ \ \ \ \
| |\     \ \ \ \ \ \ \ \ \ \
| |  \     \ \ \ \ \ \ \ \ \ \
| |    \     \ \ \ \ \ \ \ \ \ \
| |      \    | | | | | | | | | |
| |        \  | | | | | | | | | |
o o o o o o | | | | | | | | | | |    gradients/{Mean_grad/Shape_1,Mean_grad/Prod_1,Const,Mean_grad/Const,Mean_grad/Maximum/y,Shape}
| | |/ / / / / / / / / / / / / /
| |/|/ /  | | | | | | | | | | |
|/|/|/    | | | | | | | | | | |
o o o o o | | | | | | | | | | |    gradients/